In [7]:
import requests as rq
import time
import pandas as pd
from pandas import json_normalize

These following functions will account for any edge cases, NULL values or Errors that come up from the Vivino API data.

In [1]:
def try_get_flavor(vintage):
    try:
        if vintage['vintage']['wine']['taste']['flavor'] is not None:
            return vintage['vintage']['wine']['taste']['flavor']
    except (TypeError, KeyError, IndexError):
        return {'group':None
                , 'stats':{'count': None, 'score': None}
                , 'primary_keywords':[{'id': None, 'name': None, 'count': None}, 
                                    {'id': None, 'name': None, 'count': None},
                                    {'id': None, 'name': None, 'count': None},
                                    {'id': None, 'name': None, 'count': None},
                                    {'id': None, 'name': None, 'count': None}]
                , 'secondary_keywords':None}
    

In [9]:
def try_get_grape(vintage):
    try:
        if vintage['vintage']['wine']['style']['varietal_name'] is not None:
            return vintage['vintage']['wine']['style']['varietal_name']
    except (TypeError, KeyError, IndexError):
        return None

In [10]:
def try_get_food(vintage):
    try:
        if vintage['vintage']['wine']['style']['food'] is not None:
            return vintage['vintage']['wine']['style']['food']
    except (TypeError, KeyError, IndexError):
        return [{'id':None, 'name':None, 'background_image':None, 'seo_name':None}]

In [11]:
def no_index(row,column):
    l = flavor_df['flavor'][row]
    if type(l) == float or column>len(l):
        return None
    else:
        return flavor_df['flavor'][row][column-1]['group']
    


In [12]:
def no_index_score(row,column):
    l = flavor_df['flavor'][row]
    if type(l) == float or column>len(l):
        return None
    else:
        return flavor_df['flavor'][row][column-1]['stats']['count']

Below is the primary script designed to extract data from Vivino. The resulting list has been curated to align with user preferences. Due to the absence of a designated termination signal for the data extraction process from Vivino, the maximum number of pages retrievable through the API had to be manually determined. Presently, the API permits extraction from 81 pages, with each page containing 25 vintages, resulting in a total allowance of 2025 wines retrievable at once.

In [24]:
vivino_url = f'https://www.vivino.com/api/explore/explore'
headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"}
total = []

for i in range(81):

    request = rq.get(vivino_url,
                    params={
            # "country_code": "US",
            # "country_codes[]":"pt",
            "region_ids[]": '25',
            "currency_code":"USD",
            # "grape_filter":"varietal",
            "min_rating":"2",
            # "order_by":"price",
            # "order":"asc",
            # 'year_range_max':'2011',
            # 'year_range_min':'2010',
            "page": i,
            "price_range_max":"50",
            "price_range_min":"0",
            # "wine_type_ids[]":"1"
                    }, headers = headers).json()['explore_vintage']['matches']

    results = [ (
        i['vintage']['id'],  ## ID number
        i["vintage"]["year"],  ##year made
        i["vintage"]["wine"]["name"],  ##wine name
        try_get_grape(i),
        i['vintage']['wine']['winery']['name'],   ##winery name
        i['vintage']['wine']['winery']['id'], ## winery id
        i["vintage"]["statistics"]["wine_ratings_average"], ##average rating
        i["vintage"]["statistics"]["ratings_count"],##number of ratings
        i["vintage"]['wine']['region']['name'],  ##region name
        i["vintage"]['wine']['region']['id'],  ## region id
        i["vintage"]['wine']['region']['country']['name'], ##Country name
        i['price']['amount'], ## price in USD
        i['vintage']['wine']['taste']['structure'], ## taste structure
        try_get_flavor(i),
        try_get_food(i)
        )
        for i in request]
    
    total.extend(results)
wines = pd.DataFrame(total, columns = ['wine_id', 'year', 'wine','grape_variety', 'winery_name', 'winery_id', 'avg_rating', \
                                       'ratings_count', 'region_name', 'region_id', 'country_name', 'price', 'taste', 'flavor', 'food_list'])



In order to conduct a thorough exploration of the API, individual vintages are accompanied by distinct flavor profiles. The following code effectively segregates these profiles and organizes them into a standalone DataFrame. Furthermore, to facilitate seamless integration of this DataFrame with others for subsequent analysis or visualization purposes, the wine_id has been reintroduced.


In [15]:
flavor_df = pd.DataFrame(data=wines['flavor'])
column=0
row=0
for j in range(1,6):
    flavor_df[f'flavor_{j}'] = ""
    flavor_df[f'flavor_{j}_score'] = ""
    
    for i in range(len(wines)):
        flavor_df.at[i, f'flavor_{j}']=no_index(i,j)
        flavor_df.at[i, f'flavor_{j}_score'] = no_index_score(i,j)
flavor_df.insert(0, 'wine_id', wines['wine_id'])
flavor_df.insert(1, 'year', wines['year'])
flavor_df.drop(['flavor'],axis=1, inplace=True)
flavor_df

,wine_id,year,flavor_1,flavor_1_score,flavor_2,flavor_2_score,flavor_3,flavor_3_score,flavor_4,flavor_4_score,flavor_5,flavor_5_score
0,175109872,2021,red_fruit,7,oak,7,earth,6,non_oak,6,spices,5
1,8824369,2014,black_fruit,60,oak,53,non_oak,41,red_fruit,32,earth,30
2,174194548,2020,None,None,None,None,None,None,None,None,None,None
3,170805604,2021,oak,279,black_fruit,202,red_fruit,189,non_oak,180,spices,150
4,167920507,2021,oak,490,black_fruit,391,non_oak,319,earth,280,spices,256
...,...,...,...,...,...,...,...,...,...,...,...,...
2020,145108154,2016,citrus_fruit,325,tree_fruit,198,tropical_fruit,102,earth,75,vegetal,67
2021,9930656,2013,oak,92,red_fruit,76,black_fruit,64,spices,61,non_oak,53
2022,4224049,2012,citrus_fruit,640,tree_fruit,499,earth,317,vegetal,154,microbio,101
2023,150349801,2017,red_fruit,215,oak,159,earth,107,non_oak,103,black_fruit,66


We continue the process of creating DataFrames for flavors such as tannin, sweetness and intensity:

In [17]:
taste_test = json_normalize(wines['taste'])
taste_test.insert(0, 'wine_id', wines['wine_id'])
taste_test.insert(1, 'year', wines['year'])
taste_test.drop(columns=['calculated_structure_count'], inplace=True)
taste_test

,wine_id,year,acidity,fizziness,intensity,sweetness,tannin,user_structure_count
0,175109872,2021,3.136872,NaN,4.951815,1.362780,3.069652,10.0
1,8824369,2014,3.765815,NaN,4.635574,1.690137,3.382170,26.0
2,174194548,2020,3.379310,NaN,4.689655,1.893793,4.000000,0.0
3,170805604,2021,3.404283,NaN,4.807787,1.581869,3.333713,367.0
4,167920507,2021,3.255934,NaN,4.612567,1.790511,3.267794,567.0
...,...,...,...,...,...,...,...,...
2020,145108154,2016,4.069370,NaN,2.967922,1.301144,NaN,67.0
2021,9930656,2013,2.026529,NaN,4.651704,2.414917,2.627097,30.0
2022,4224049,2012,4.015811,NaN,2.937798,1.255500,NaN,199.0
2023,150349801,2017,3.499718,NaN,2.950133,1.490268,2.137976,153.0


Now that we have created all of our extra DataFrames, to consolidate and clean up our initial data, we will drop the taste, flavor and food_list from the original data before we create our CSVs.

In [19]:
wines.drop(['taste','flavor','food_list'],axis=1, inplace=True)
wines

,wine_id,year,wine,grape_variety,winery_name,winery_id,avg_rating,ratings_count,region_name,region_id,country_name,price
0,175109872,2021,M By Merus Cabernet Sauvignon,Cabernet Sauvignon,Merus,1198,4.5,5,Napa Valley,25,United States,39.999167
1,8824369,2014,Perry's Blend,Bordeaux Blend,Juslyn,406,4.3,38,Spring Mountain District,73,United States,44.990000
2,174194548,2020,Edict Napa Valley Proprietary Red,Bordeaux Blend,Edict,46553,4.5,9,Napa Valley,25,United States,28.990000
3,170805604,2021,Cabernet Sauvignon,Cabernet Sauvignon,Bella Union,115511,4.4,12,Napa Valley,25,United States,49.640000
4,167920507,2021,Cabernet Sauvignon,Cabernet Sauvignon,The Prisoner,89393,4.4,1205,Napa Valley,25,United States,42.440000
...,...,...,...,...,...,...,...,...,...,...,...,...
2020,145108154,2016,Miller Ranch Sauvignon Blanc,Sauvignon Blanc,Silverado Vineyards,1205,3.8,400,Napa Valley,25,United States,12.650000
2021,9930656,2013,Ink Grade Vineyard Zinfandel,Zinfandel,Heitz Cellar,11538,3.8,392,Napa Valley,25,United States,28.920000
2022,4224049,2012,Sauvignon Blanc,Sauvignon Blanc,Emmolo,5528,3.9,391,Napa Valley,25,United States,20.990000
2023,150349801,2017,Pinot Noir,Pinot Noir,Artesa,1534,3.9,382,Los Carneros,96,United States,18.990000


In [82]:
## READY FOR EXECUTION
## Back uppppp
row=0
flavor_df = pd.DataFrame(data=wines['flavor'].tolist())
## Defining the columns
flavor_df['group_stat']=''
flavor_df['kw1'] = ""
flavor_df['kw1_score'] = ""
flavor_df['kw2'] = ""
flavor_df['kw2_score'] = ""
flavor_df['kw3'] = ""
flavor_df['kw3_score'] = ""
flavor_df['kw4'] = ""
flavor_df['kw4_score'] = ""
flavor_df['kw5'] = ""
flavor_df['kw5_score'] = ""

for i in range(len(wines)):
    row+=1
    try:
        
        flavor_df['group_stat'][i] = flavor_df['stats'][i]['score']
        flavor_df['kw1'][i] = no_float(i,j)
        flavor_df['kw1_score'][i] = flavor_df['primary_keywords'][i][0]['count']
        flavor_df['kw2'][i] = flavor_df['primary_keywords'][i][1]['name']
        flavor_df['kw2_score'][i] = flavor_df['primary_keywords'][i][1]['count']
        flavor_df['kw3'][i] = flavor_df['primary_keywords'][i][2]['name']
        flavor_df['kw3_score'][i] = flavor_df['primary_keywords'][i][2]['count']
        flavor_df['kw4'][i] = flavor_df['primary_keywords'][i][3]['name']
        flavor_df['kw4_score'][i] = flavor_df['primary_keywords'][i][3]['count']
        flavor_df['kw5'][i] = flavor_df['primary_keywords'][i][4]['name']
        flavor_df['kw5_score'][i] = flavor_df['primary_keywords'][i][4]['count']
    except IndexError:
        flavor_df['group_stat'][i]=None
        flavor_df['kw1'][i]=None
        flavor_df['kw1_score'][i]=None
        flavor_df['kw2'][i]=None
        flavor_df['kw2_score'][i]=None
        flavor_df['kw3'][i]=None
        flavor_df['kw3_score'][i]=None
        flavor_df['kw4'][i]=None
        flavor_df['kw4_score'][i]=None
        flavor_df['kw5'][i]=None
        flavor_df['kw5_score'][i]=None
        
flavor_df.insert(0, 'wine_id', wines['wine_id'])
flavor_df.drop(['stats','primary_keywords','secondary_keywords'],axis=1, inplace=True)
flavor_df

,wine_id,group,group_stat,kw1,kw1_score,kw2,kw2_score,kw3,kw3_score,kw4,kw4_score,kw5,kw5_score
0,173744538,red_fruit,14974,None,62,strawberry,29,raspberry,22,red fruit,15,red cherry,8
1,164415516,oak,11480,None,67,vanilla,23,butter,19,coconut,4,caramel,3
2,168607052,black_fruit,6666,None,18,blackberry,17,dark fruit,13,plum,9,ripe blackberry,3
3,162923665,oak,19664,None,91,vanilla,45,chocolate,31,tobacco,12,cola,7
4,149904981,oak,7499,None,25,vanilla,17,tobacco,15,chocolate,8,dark chocolate,5
5,3506246,red_fruit,6518,None,23,raspberry,12,red fruit,10,strawberry,9,cranberry,4
6,1564027,tree_fruit,10000,None,42,pear,24,green apple,21,asian pear,6,baked apple,2
7,2347586,red_fruit,5746,None,19,cherry,19,strawberry,9,red fruit,3,ripe strawberry,2
8,11517987,red_fruit,5746,None,19,raspberry,19,strawberry,9,red fruit,3,ripe strawberry,2
9,172361342,citrus_fruit,9100,None,42,grapefruit,16,lemon,12,lemon zest,6,lime,4


Finally to turn each DataFrame into a CSV that will assist in the next stages of analysis:

In [21]:
flavor_df.to_csv('flavor.csv', index=False)
taste_test.to_csv('taste.csv', index=False)
wines.to_csv('wines.csv', index=False)